#Taller 3 Árboles de decisión
## Carlos Fernando Gonzalez Sarmiento

# Árbol de decisión

Según el siguiente árbol de decisión, analice y responda:

![Obama-Clinton decision tree](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/obama_clinton_tree.jpg)

## 1.1 ¿Qué son las observaciones? ¿Cuántas observaciones hay?


Las observaciones se pueden tomar como un chance ya que dando un ejemplo se está hablando de el numero de votos que obtuvo un candidato a comparación del otro y si este ganó en los condados, de este modo se pueden tomar como observaciones el voto de cada condado para cada candidato.

Ahora bien, cuando se habla de cuantas observaciones existen es la suma de los votos de los condados para Obama que es 1031 y Hillary 1210.

## 1.2 ¿Cuál es la variable de respuesta?


La variable respuesta es el candidato ganador según el numero de votos de los condados.

## 1.3 ¿Cuáles son las características?¿Cuál es la característica más predictiva?


Las características son:

*   Porcentaje de personas negras en el condado.
*   Tasa de graduación de la escuela secundaria
*   Donde se encuentra el condado
*   Su hogar ganaba mas de $30.000 para el año 2000
*   Densidad de la población del condado
*   ¿Cree que Bush venció por mas del 16.5% a Kerry?

La característica mas predictiva es el porcentaje de personas negras en el condado, ya que de allí es donde inicia el arbol de decision y donde mas se puede subdividir la muestra, al tener un menor RMSE.




## 1.4 ¿Cuál es la predicción de clase para el siguiente condado con los siguientes datos: 15% de afroamericanos, 90% de tasa de graduación en la escuela secundaria, ubicado en el Sur, alta pobreza, alta densidad de población?


Gana Obama

## 1.5 ¿Cuál es el chance predicha para ese mismo condado anteriormente descrito?

185 a 36

## SVM


Para esta tarea consideramos un conjunto de observaciones sobre una serie de variedades de vino tinto y blanco que implican sus propiedades químicas y su clasificación por parte de los catadores. La industria del vino ha experimentado un gran crecimiento en los últimos tiempos debido al aumento del consumo social. El precio del vino depende de un concepto bastante abstracto de apreciación del vino por parte de los catadores, cuya opinión puede tener un alto grado de variabilidad. El precio del vino depende en cierta medida de este factor tan volátil. Otro factor clave en la certificación y evaluación de la calidad del vino son las pruebas fisicoquímicas, que se realizan en laboratorio y tienen en cuenta factores como la acidez, el nivel de pH, la presencia de azúcar y otras propiedades químicas. Para el mercado del vino, sería interesante que la calidad humana de la cata pudiera relacionarse con las propiedades químicas del vino para que el proceso de certificación y evaluación de la calidad estuviera más controlado.

Se dispone de dos conjuntos de datos, uno de los cuales se refiere al vino tinto y cuenta con 1.599 variedades diferentes, y el otro, al vino blanco, con 4.898 variedades. Todos los vinos se producen en una zona concreta de Portugal. Se recogen datos sobre 12 propiedades diferentes de los vinos, una de las cuales es la calidad, basada en datos sensoriales, y el resto son propiedades químicas de los vinos, como la densidad, la acidez, el contenido de alcohol, etc. Todas las propiedades químicas de los vinos son variables continuas. La calidad es una variable ordinal con una clasificación posible de 1 (peor) a 10 (mejor). Cada variedad de vino es catada por tres catadores independientes y la clasificación final asignada es la mediana de la clasificación dada por los catadores.

Se espera que un modelo predictivo desarrollado a partir de estos datos sirva de orientación a los viñedos en cuanto a la calidad y el precio que se espera obtener de sus productos sin depender en gran medida de la volatilidad de los catadores.


In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://raw.githubusercontent.com/prof-tatiana-gutierrez/DatosMD/master/Wine_data_red.csv')
data_w = pd.read_csv('https://raw.githubusercontent.com/prof-tatiana-gutierrez/DatosMD/master/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
6274,8.2,0.885,0.20,1.4,0.086,7.0,31.0,0.99460,3.11,0.46,10.000000,5,red
2544,6.9,0.320,0.30,1.8,0.036,28.0,117.0,0.99269,3.24,0.48,11.000000,6,white
5479,12.3,0.500,0.49,2.2,0.089,5.0,14.0,1.00020,3.19,0.44,9.600000,5,red
4734,5.5,0.160,0.31,1.2,0.026,31.0,68.0,0.98980,3.33,0.44,11.633333,6,white
2156,7.9,0.280,0.32,3.6,0.038,9.0,76.0,0.99200,3.05,0.31,11.700000,4,white


In [4]:
data.quality.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [5]:
data.type.value_counts()

white    4898
red      1599
Name: type, dtype: int64

# Ejercicio 2.1

Mostrar la tabla de frecuencias de la calidad por tipo de vino.

In [6]:
pd.crosstab(index=data['type'],columns=data['quality'])

quality,3,4,5,6,7,8,9
type,,,,,,,
red,10,53,681,638,199,18,0
white,20,163,1457,2198,880,175,5


In [7]:
pd.crosstab(index=data['quality'],columns=data['type'])

type,red,white
quality,,
3,10,20
4,53,163
5,681,1457
6,638,2198
7,199,880
8,18,175
9,0,5


# Ejercicio 2.2

Teniendo en cuenta todas las variables:

* Crear un objetivo binario para cada tipo de vino, haciendo la transformación necesaria para ingresar la información al modelo.
* Crear una SVM que prediga el tipo de vino
* Evalúe con F1, AUC-ROC y log-loss


In [8]:
data['type']= data['type'].map({'red':1,'white':0})

In [9]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,0
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,0
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,0
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1


In [14]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[["3", " 4", 
                                      "5", "6", "7", 
                                      "8", "9"]])
encoder.fit(data[["quality"]])
data["cal-enc"] = encoder.transform(data[["quality"]])

In [15]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,cal-enc
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,0,3.0
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,0,3.0
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,0,3.0
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0,3.0
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,1,2.0
6493,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1,3.0
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1,3.0
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,1,2.0


In [16]:
from sklearn import svm
from sklearn.model_selection import train_test_split
X = np.array(data.drop(['type','quality'], 1))
y = np.array(data['type'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=55)
svc_clf = svm.SVC(probability=True)
svc_clf.fit(X_train,y_train)
y_pred=svc_clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
from sklearn import metrics
a=[metrics.f1_score(y_test,y_pred),metrics.roc_auc_score(y_test, y_pred,average=None),metrics.log_loss(y_test,y_pred)]
b=['F1','AU-ROC','log-loss']
df=pd.DataFrame()
df['Metrica']=b
df['Valor']=a
df

,Metrica,Valor
0,F1,0.863309
1,AU-ROC,0.890887
2,log-loss,2.019199


# Ejercicio 2.3

* Probar la SVM utilizando los diferentes kernels ('poly', 'rbf', 'sigmoid')
* Evalúe con F1, AUC-ROC y log-loss


In [25]:
svc_clf1 = svm.SVC(kernel='poly',probability=True);svc_clf2 = svm.SVC(kernel='rbf',probability=True);svc_clf3 = svm.SVC(kernel='sigmoid',probability=True)
svc_clf1.fit(X_train,y_train);svc_clf2.fit(X_train,y_train);svc_clf3.fit(X_train,y_train)
y_pred1=svc_clf1.predict(X_test);y_pred2=svc_clf2.predict(X_test);y_pred3=svc_clf3.predict(X_test)
a1=[metrics.f1_score(y_test,y_pred1),metrics.f1_score(y_test,y_pred2),metrics.f1_score(y_test,y_pred3)]
a2=[metrics.roc_auc_score(y_test, y_pred1,average=None),metrics.roc_auc_score(y_test, y_pred2,average=None),metrics.roc_auc_score(y_test, y_pred3,average=None)]
a3=[metrics.log_loss(y_test,y_pred1),metrics.log_loss(y_test,y_pred2),metrics.log_loss(y_test,y_pred3)]
b=['ply','rbf','sigmoid']
df2=pd.DataFrame()
df2['kernel']=b
df2['f1']=a1
df2['AUC-ROC']=a2
df2['log-loss']=a3
df2

,kernel,f1,AUC-ROC,log-loss
0,ply,0.844944,0.895235,2.444308
1,rbf,0.863309,0.890887,2.019199
2,sigmoid,0.507659,0.679868,7.970581


Se evidencia que 'rbf' posee la metrica f1 mas alta y la perdida logaritmica mas baja de los 3 kernels, a pesar de que el AUC-ROC es mas alto en el kernel 'ply' es por decimas, nada significativo respecto al valor de rbf, po ello se escoge el kernel 'rbf' coo mejor svm.

# Ejercicio 2.4
* Utilizando el mejor SVM del punto anterior encontrar los parámetros que da el mejor rendimiento teniendo en cuenta los siguientes hiperparámetros
'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]
* Evalúe cada modelo con F1, AUC-ROC y log-loss

Nota: Para los valores de C y gamma, deben realizar todas las posibles combinaciones entre estas dos variables.

In [26]:
a=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df1=pd.DataFrame()
df1['C']=a
df1['gamma']=a
df1['f1']=a
df1['AUC-ROC']=a
df1['log-loss']=a

In [27]:
from sklearn import metrics
c_est = [0.1, 1, 10, 100, 1000]
gam= [0.01, 0.001, 0.0001]

k=0
for i in c_est:
  for j in gam:
    svc_clf = svm.SVC(C=i,gamma=j,probability=True,kernel='rbf')
    svc_clf.fit(X_train,y_train)
    y_pred=svc_clf.predict(X_test)
    df1.iloc[[k],0] = i 
    df1.iloc[[k],1] = j
    df1.iloc[[k],2]=metrics.f1_score(y_test,y_pred)
    df1.iloc[[k],3]=metrics.roc_auc_score(y_test, y_pred,average=None)
    df1.iloc[[k],4]=metrics.log_loss(y_test,y_pred)
    k=k+1

Tabla de todas las combinaciones ordenada de la mejor a la peor según las metricas

In [21]:
df1.sort_values(['f1','AUC-ROC','log-loss'],ascending=[False,False,True])

,C,gamma,f1,AUC-ROC,log-loss
13,1000.0,0.0010,0.984547,0.990693,0.247974
14,1000.0,0.0001,0.984340,0.986058,0.247972
10,100.0,0.0010,0.977778,0.984055,0.354247
9,100.0,0.0100,0.973568,0.984265,0.425098
12,1000.0,0.0100,0.973451,0.982720,0.425098
6,10.0,0.0100,0.961969,0.971658,0.602219
11,100.0,0.0001,0.924138,0.939350,1.169011
7,10.0,0.0010,0.914943,0.933590,1.310710
3,1.0,0.0100,0.892019,0.913678,1.629530
8,10.0,0.0001,0.884706,0.908586,1.735804


Se obtuvo que la combinacion de mejores metricas en 2 casos fue la de C:1000, gamma: 0.001 con un kernel rbf y metricas f1:0.984547, AUC-ROC: 0.990693 y log-loss:0.247974 esta metrica aunque no haya sido mas pequeña en las combinaciones es la segunda mas pequeña por una diferencia decimal bastante pequeña.

# Ejercicio 2.5

Compare los resultados con todos los anteriores modelos y eliga ¿Cuál algoritmo con cuáles hiperparámetros es el que tiene mejor rendimiento según la puntuación F1, AUC-ROC y log-loss?

## Modelo pre-determinado
C:1, Kernel:'rbf'

In [22]:
df

,Metrica,Valor
0,F1,0.863309
1,AU-ROC,0.890887
2,log-loss,2.019199


## Modelos con diferente kernel

In [23]:
df2

,kernel,f1,AUC-ROC,log-loss
0,ply,0.844944,0.895235,2.444308
1,rbf,0.863309,0.890887,2.019199
2,sigmoid,0.507659,0.679868,7.970581


## Modelos cambiando C y gamma co  kernel rbf

In [24]:
df1.sort_values(['f1','AUC-ROC','log-loss'],ascending=[False,False,True])

,C,gamma,f1,AUC-ROC,log-loss
13,1000.0,0.0010,0.984547,0.990693,0.247974
14,1000.0,0.0001,0.984340,0.986058,0.247972
10,100.0,0.0010,0.977778,0.984055,0.354247
9,100.0,0.0100,0.973568,0.984265,0.425098
12,1000.0,0.0100,0.973451,0.982720,0.425098
6,10.0,0.0100,0.961969,0.971658,0.602219
11,100.0,0.0001,0.924138,0.939350,1.169011
7,10.0,0.0010,0.914943,0.933590,1.310710
3,1.0,0.0100,0.892019,0.913678,1.629530
8,10.0,0.0001,0.884706,0.908586,1.735804


Para todos los modelos incluyendo el modelo pre-determinado, en el que se usan diferentes kernel y en el que se cambien los hiperparametros el mejor modelo en cuestión de mestricas f1 y AUC-ROC es el modelo con kernel gamma: 0.001, C: 1000 y kernel rbf teniendo un f1 de 0.984547 y un AUC-ROC de 0.990693 como se mencionó anteriormente la perdida logaritmica o log-loss es el segundo mas pequeño por una pequeña diferencia con el primero de 0.000002 con el valor de 0.247974 para el modelo con los parametros ya mencionados pero a pesar de esto y al ser una diferencia bastante minima es el escogido al tener mejore metricas.